In [25]:
#Creating the functions
import pandas as pd

#load in the dataframes for functions
lemmaDF = pd.read_csv("Dictionary_Dataframes/greek_words.csv", sep = "\t")
del lemmaDF['Unnamed: 0']
#outputs list of matching IDs or an empty list
def get_id(word):
    #search the lemmaDF for the word
    word = word.lower()
    output = []
    #this is the dataframe of all rows with the word
    df = lemmaDF.loc[lemmaDF["bare_text"] == word]
        #iterrows works, but the others dont. ok whatever
    for index, row in df.iterrows():
        if(row["id"] not in output):
            output.append(row["id"])
    return(output)


#outouts dictionary of ids to the valid words 
def get_words_help(ids):
    output = {}
    if(type(ids) == list):
        for id in ids:
            curOut = []
            df = lemmaDF.loc[lemmaDF["id"] == id]
            for index, row in df.iterrows():
                #can add a checker for POS or add info about POS
                curOut.append(row["bare_text"])
            output[id] = curOut
        return output
       #for each id in the list, grab all the words with matching ID from parses and put in "outputs"
       #outputs is dict of ID --> parses with ID 
    else:
        print("Something went wrong...")
        return {}
    #enter FPP

def get_words(word):
    ids = get_id(word)
    return get_words_help(ids)
def check_enclitic(row, text):
    print("out of date")
def get_def(id):
    output = ""
    df = lemmaDF.loc[lemmaDF["id"] == id]
    for index, row in df.iterrows():
        output = (row["definition"])
    return output
def get_lang(id):
    #returns ltn or grk
    output = ""
    df = lemmaDF.loc[lemmaDF["id"] == id]
    for(index, row) in df.iterrows():
        output = row["lang_id"]
    if(output == 3 ):
        output = "ltn"
    else:
        output = "grk"
    return output
def get_all_ids():
    #returns a list of all noun ids
    output = []
    #for every id in lemmas, add it to output
    for index, row in lemmaDF.iterrows():
        out = row["id"]
        #if an int, add it to the output
        if(type(out) == type(1) and out not in output):
            output.append(out)
    return output

def make_dict_csv(ids, outputFile = "outputs/nounsDictionary.csv", allowNulls = True):
    #ids is the list of ids for all words - check for nouns in here 
    FPP = []
    POS = []
    DICTENTRY = []
    SOURCE = []
    for id in ids:
        pos = "" 
        fpp = ""
        dictEntry = ""
        source = "morpheus"
        text = {}
        df = lemmaDF.loc[lemmaDF["id"] == id]
        for i in range(10):
                if(not i in text):
                    text[i] = "NULL"
        #if no words in parses, we can't make an entry for it -- current outlook
        if(df.empty):
            continue
        for index, row in df.iterrows():
            #preset text to be all null
            rowText = row["bare_text"]
            morph = row["morph_code"]
            #noun case
                #check required to prevent substantive adj from outputting as nouns 
            if (row["morph_code"][0] == "n" and (pos == "" or pos == "noun")):
                pos = "noun"
                text[2] = row["morph_code"][6] 
                #make the nom, singular --> text[0]
                #make the gen, singular --> text[1]
                if((row["morph_code"][2] == "s") and (row["morph_code"][7] == "g")):
                    if(text[1] == "NULL" or len(row["bare_text"])<len(text[1])):
                        text[1] = row["bare_text"] 
                #hold the plural genative just in case
                if(morph[2] == "s" and (morph[7] == "g")):
                    if(text[6] == "NULL" or len(row["bare_text"])<len(text[6])):
                        text[6] = row["bare_text"] 
            #adj case 
            if (row["morph_code"][0] == "a"):
                pos = "adj"
                #for each, need the singular, positive, nominative - without enclitics
                if((row["morph_code"][2] == "s")
                and (row["morph_code"][7] == "n") 
                and (row["morph_code"][6] == "m")
                and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                    if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                        text[0] = row["bare_text"]
                
                if((row["morph_code"][2] == "s")
                and (row["morph_code"][7] == "n") 
                and (row["morph_code"][6] == "f")
                and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                    if(text[1] == "NULL" or len(row["bare_text"])<len(text[1])):
                        text[1] = row["bare_text"]

                if((row["morph_code"][2] == "s")
                and (row["morph_code"][7] == "n") 
                and (row["morph_code"][6] == "n")
                and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                    if(text[2] == "NULL" or len(row["bare_text"])<len(text[2])):
                        text[2] = row["bare_text"]
            #verb case - need t morph code for the 4PP
            if(row["morph_code"] == "t-sr-pmn-") and (text[3] == "NULL" or len(row["bare_text"])<len(text[3])):
                text[3] = row["bare_text"] 
            #backup 4PP for verbs stored in text[6] - for weird words like futurus 
            if(row["morph_code"] == "t-sf-amn-") and (text[6] == "NULL" or len(row["bare_text"])<len(text[6])):
                text[6] = row["bare_text"] 
            if((row["morph_code"][0] == "v")): 
                pos = "verb"
                kind = "regular"
                morph = row["morph_code"]
                curWord = row["bare_text"]
                #make 3 cases: regular, deponent, and inpersonal - may be more ask Celano
                #0: present first person singular active indicative: v1spia---
                #1: present active infinitive: v--pna---
                #2: perfect first person singular active indicative: v1sria---
                #3: t-sr-pmn- is the morph code --> has 3PP if deponent 
                #4: present third person singular active indicative - for impersonal verbs
                #5: grab stuff with deponent voice tag v1spip---
            
                if(morph) == "v1spia---":
                    if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                        text[0] = row["bare_text"]                    
                if(morph) == "v--pna---":
                    if(text[1] == "NULL" or len(row["bare_text"])<len(text[1])):
                        text[1] = row["bare_text"]
                if(morph) == "v1sria---":
                    if(text[2] == "NULL" or len(row["bare_text"])<len(text[2])):
                        text[2] = row["bare_text"]
                if(morph) == "v3spia---":
                    if(text[4] == "NULL" or len(row["bare_text"])<len(text[4])):
                        text[4] = row["bare_text"]
                if(morph) == "v1spip---":
                    if(text[5] == "NULL" or len(row["bare_text"])<len(text[5])):
                        text[5] = row["bare_text"]
             #all other cases - simplest, just has 1 form 
            if(morph[0] == "d"):
                pos = "adv"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                            text[0] = row["bare_text"]
            if(morph[0] == "c"):
                pos = "conj"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                     text[0] = row["bare_text"]
            if(morph[0] == "r"):
                pos = "adpostion"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "p"):
                pos = "pronoun"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "m"):
                pos = "numeral"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "i"):
                pos = "interjection"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "e" ):
                pos = "exclaimation"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "u"):
                pos = "punctuation"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]  
        #set FPP to text[0]
        df2 = lemmaDF.loc[lemmaDF["id"] == id]
        if(pos == "noun"):
            for index, row in df2.iterrows():
                text[0] = row["bare_text"]
            if(text[1] == "NULL"):
                text[1] = text[6]    
            dictEntry = text[0] + ", " + text[1] + ", " + text[2] 
            fpp = text[0]
            if("NULL" not in dictEntry or allowNulls):
                FPP.append(fpp)
                POS.append(pos)
                DICTENTRY.append(dictEntry)
        elif(pos == "verb"):
            #change which type by checking if we have a normal FPP
            if(text[0]!= "NULL"):
                kind = "regular"
            elif(text[5] != "NULL"):
                kind = "deponent"
            else:
                kind = "impersonal"
                #nested switch block! Yay!
            #fix edge case - 4PP is weird
            if(text[3] == "NULL"):
                    text[3] = text[6]
            match(kind):
                case "regular":
                    fpp = text[0]
                    dictEntry = text[0] + ", " + text[1] + ", "+ text[2] + ", " + text[3]
                case "deponent":
                    fpp = text[5]
                    dictEntry = text[5] + ", " + text[1] + ", " + text[3]
                case "impersonal":
                    fpp = text[4]
                    dictEntry = text[4] + ", " + text[1] +", " + text[2]+  ", "+ text[3]
            if(fpp == "NULL"):
                for index, row in df2.iterrows():
                    fpp = row["bare_text"]
            if("NULL" not in dictEntry or allowNulls):
                FPP.append(fpp)
                POS.append(pos)
                DICTENTRY.append(dictEntry)
        elif(pos == "adj"):
            for index, row in df2.iterrows():
                text[0] = row["bare_text"]
            fpp = text[0]
            dictEntry = text[0] + ", " + text[1] + ", " + text[2] 
            if("NULL" not in dictEntry or allowNulls):
                FPP.append(fpp)
                POS.append(pos)
                DICTENTRY.append(dictEntry)
        else:
            for index, row in df2.iterrows():
                text[0] = row["bare_text"]
            fpp = text[0]
            dictEntry = text[0] + " indecl."
            if("NULL" not in dictEntry or allowNulls):
                FPP.append(fpp)
                POS.append(pos)
                DICTENTRY.append(dictEntry)

    #end of function stuff
    out_map = {
        "FPP": FPP,
        "POS": POS,
        "Dictionary Entry": DICTENTRY,
    }
    print(len(FPP))
    print(len(POS))
    print(len(DICTENTRY))   
    output = pd.DataFrame(out_map)
    output.to_csv(outputFile, sep = "\t")

            


        
        
#function from word -> dictionary entry
def get_dict(word, show_def = False):
    ids = get_id(word)
        #list of dictionary entries 
    output = {}
    for id in ids:
        lang = get_lang(id)
        if(lang == "grk"):
            continue
        pos = "" 
        dictEntry = ""
        rowText = ""
        morph = ""
        text = {}
        definition = get_def(id)
        df = lemmaDF.loc[lemmaDF["id"] == id]
        for i in range(10):
                if(not i in text):
                    text[i] = "NULL"
        for index, row in df.iterrows():
            #preset text to be all null
            rowText = row["bare_text"]
            morph = row["morph_code"]
            #noun case
                #check required to prevent substantive adj from outputting as nouns 
            if (row["morph_code"][0] == "n" and (pos == "" or pos == "noun")):
                pos = "noun"
                text[2] = row["morph_code"][6] 
                #make the nom, singular --> text[0]
                if((row["morph_code"][2] == "s") and (row["morph_code"][7] == "n")):
                    #have to hard-code around que enclitic 
                    if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):

                        text[0] = row["bare_text"]
                #make the gen, singular --> text[1]
                if((row["morph_code"][2] == "s") and (row["morph_code"][7] == "g")):
                    if(text[1] == "NULL" or len(row["bare_text"])<len(text[1])):
                        text[1] = row["bare_text"]
            
            #adj case 
            if (row["morph_code"][0] == "a"):
                pos = "adj"
                #for each, need the singular, positive, nominative - without enclitics
                if((row["morph_code"][2] == "s")
                and (row["morph_code"][7] == "n") 
                and (row["morph_code"][6] == "m")
                and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                    if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                        text[0] = row["bare_text"]
                
                if((row["morph_code"][2] == "s")
                and (row["morph_code"][7] == "n") 
                and (row["morph_code"][6] == "f")
                and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                    if(text[1] == "NULL" or len(row["bare_text"])<len(text[1])):
                        text[1] = row["bare_text"]

                if((row["morph_code"][2] == "s")
                and (row["morph_code"][7] == "n") 
                and (row["morph_code"][6] == "n")
                and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                    if(text[2] == "NULL" or len(row["bare_text"])<len(text[2])):
                        text[2] = row["bare_text"]
            #verb case - need t morph code for the 4PP
            if(row["morph_code"] == "t-sr-pmn-") and (text[3] == "NULL" or len(row["bare_text"])<len(text[3])):
                text[3] = row["bare_text"] 
            #backup 4th PP for verbs stored in text[6]
            if(row["morph_code"] == "t-sf-amn-") and (text[6] == "NULL" or len(row["bare_text"])<len(text[6])):
                text[6] = row["bare_text"] 
            if((row["morph_code"][0] == "v")): 
                pos = "verb"
                kind = "regular"
                morph = row["morph_code"]
                curWord = row["bare_text"]
                #make 3 cases: regular, deponent, and inpersonal - may be more ask Celano
                #0: present first person singular active indicative: v1spia---
                #1: present active infinitive: v--pna---
                #2: perfect first person singular active indicative: v1sria---
                #3: t-sr-pmn- is the morph code --> has 3PP if deponent 
                #4: present third person singular active indicative - for impersonal verbs
                #5: grab stuff with deponent voice tag v1spip---
                if(morph) == "v1spia---":
                    if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                        text[0] = row["bare_text"]                    
                if(morph) == "v--pna---":
                    if(text[1] == "NULL" or len(row["bare_text"])<len(text[1])):
                        text[1] = row["bare_text"]
                if(morph) == "v1sria---":
                    if(text[2] == "NULL" or len(row["bare_text"])<len(text[2])):
                        text[2] = row["bare_text"]
                if(morph) == "v3spia---":
                    if(text[4] == "NULL" or len(row["bare_text"])<len(text[4])):
                        text[4] = row["bare_text"]
                if(morph) == "v1spip---":
                    if(text[5] == "NULL" or len(row["bare_text"])<len(text[5])):
                        text[5] = row["bare_text"]                    
            #all other cases
            if(morph[0] == "d"):
                pos = "adv"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                            text[0] = row["bare_text"]
            if(morph[0] == "c"):
                pos = "conj"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                     text[0] = row["bare_text"]
            if(morph[0] == "r"):
                pos = "adpostion"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "p"):
                pos = "pronoun"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "m"):
                pos = "numeral"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "i"):
                pos = "interjection"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "e" ):
                pos = "exclaimation"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "u"):
                pos = "punctuation"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
        #preventing errors <3 
        if(type(definition) != type("str") or len(definition) == 0):
            definition = "No Definiton"
            #case for when FPP isn't found in parses
        if(text[0] == "NULL"):
            df2 = lemmaDF.loc[lemmaDF["id"] == id]
            for index, row in df2.iterrows():
                text[0] = row["bare_text"]
        #make dictionary entry string
        match(pos):
            #make sure each of those exists before trying to output
            case "noun":
                if(show_def):
                    dictEntry = pos + ": " + text[0] + ", " + text[1] + ", " + text[2] + ". - " + (definition)
                else:
                    dictEntry = pos + ": " + text[0] + ", " + text[1] + ", " + text[2] 
            case "adj":
                if(show_def):
                    dictEntry = pos + ": " + text[0] + ", " + text[1] + ", " + text[2] + " - " + definition
                else:
                    dictEntry = pos + ": " + text[0] + ", " + text[1] + ", " + text[2] 
            case "verb":
                #change which type by checking if we have a normal FPP
                if(text[0]!= "NULL"):
                    kind = "regular"
                elif(text[5] != "NULL"):
                    kind = "deponent"
                else:
                    kind = "impersonal"
                    #nested switch block! Yay!
                #solve the edge case! 
                if(text[3] == "NULL"):
                    text[3] = text[6]
                match(kind):
                    case "regular":
                        if(show_def):
                            dictEntry = pos + ": " + text[0] + ", " + text[1] + ", " + text[2] + ", " + text[3] + "- " + definition
                        else:
                            dictEntry = pos + ": " + text[0] + ", " + text[1] + ", "+ text[2] + ", " + text[3]
                    case "deponent":
                        if(show_def):
                            dictEntry = pos + ": " + text[5] + ", " + text[1] + ", " + text[3] + "- " + definition
                        else:
                            dictEntry = pos + ": " + text[5] + ", " + text[1] + ", " + text[3]
                    case "impersonal":
                        if(show_def):
                            dictEntry = pos + ": " + text[4] + ", " + text[1] +", " + text[2] + ", " + text[3] + "- " + definition
                        else:
                            dictEntry = pos + ": " + text[4] + ", " + text[1] +", " + text[2]+  ", "+ text[3]
            case other:
                dictEntry = pos + ": " + text[0]
                if(show_def):
                    dictEntry += " " + definition 
        output[id] = (dictEntry)
    return output

In [28]:
# #Counting my errors 
# dictionary_csv = pd.read_csv("outputs/Greek_dictionary1.1.csv", sep = "\t")
# del dictionary_csv["Unnamed: 0"]

# def get_null():
#     output = 0
#     for index, row in dictionary_csv.iterrows():
#         if("NULL" in row["Dictionary Entry"]):
#             output += 1
#     return output
# def get_non_null():
#     output = 0
#     for index, row in dictionary_csv.iterrows():
#         if("NULL" not in row["Dictionary Entry"]):
#             output += 1
#     return output
# print("Number of NULL entries: " + str(get_null()))
# print("Number of non-NULL entries: " + str(get_non_null())) 

FileNotFoundError: [Errno 2] No such file or directory: 'outputs/Greek_dictionary1.1.csv'

In [29]:
#new text block for getForm 
#load in the dataframes for functions
lemmaDF = pd.read_csv("Dictionary_Dataframes/greek_words.csv", sep = "\t")
del lemmaDF['Unnamed: 0']

def get_POS(id):
    #returns the part of speech of the word
    output = []
    df = lemmaDF.loc[lemmaDF["id"] == id]
    for(index, row) in df.iterrows():
        out = row["morph_code"][0]
        if out not in output:
            output.append(out)
    return output
def get_form(word = "NULL", id = "NULL", case = "NULL",
              number = "NULL", gender = "NULL", mood = "NULL", 
              person = "NULL", tense = "NULL", voice = "NULL",
                degree = "NULL", wanted_pos = "NULL"):
    #check if word is null - use ID in that case
    if(word == "NULL" and id == "NULL"):
        print("Please enter a word or an ID")
        return "ERROR: Please enter a word or an ID"
    if(word!= "NULL" and id !="NULL"):
            print("Please enter a word or an ID, not both")
            return "ERROR: Please enter a word or an ID, not both"
    if(word != "NULL"):
        ids = get_id(word)
        if(len(ids) == 0):
            print("No matching words found")
            return "ERROR: No matching words"
    if(id != "NULL"):
        ids = [id]
    for id in ids:
        #if you find a valid output in one id, you're good. If not, we continue on 
        output = ""
        posTag = "---------"
        pos = get_POS(id)
        #if the word is a noun
        if("n" in pos):
            #noun 
            #case
            if (case == "nominative"):
                case = "n"
            if (case == "genative"):
                case = "g"
            if (case == "dative"):
                case = "d"
            if (case == "accusative"):
                case = "a"
            if (case == "ablative"):
                case = "b"
            if (case == "locative"):
                case = "l"
            if (case == "vocative"):
                case = "v" 
            
            #number
            if (number == "singular"):
                number = "s"
            if (number == "plural"):
                number = "p"
            
            #gender 
            if(gender == "masculine"):
                gender = "m"
            if (gender == "feminine"):
                gender = "f"
            if (gender == "neuter"):
                gender = "n"
           
            #word assigned to output if it exists
            if (case != "NULL"):
                # include case in search
                posTag = posTag[:7] + case + posTag[8:]
            # number
            if (number != "NULL"):
                posTag = posTag[:2] + number + posTag[3:]
            # gender 
            if(gender != "NULL"):
                posTag = posTag[:6] + gender + posTag[7:]
        if("v" in pos):
            #verb
                #mood
            if(mood == "indicative"):
                mood = "i"
            if(mood == "subjunctive"):
                mood = "s"
            if(mood == "imperative"):
                mood = "m"
            if(mood == "infinitive"):
                mood = "n"
            if(mood == "participle"):
                mood = "p"
            if(mood == "gerund"):
                mood = "d"
            if(mood == "gerundive"):
                mood = "g"
            
                #person
            if(person == "first"):
                person = "1"
            if(person == "second"):
                person = "2"
            if(person == "third"):
                person = "3"
            
                #tense
            if(tense == "present"):
                tense = "p"
            if(tense == "imperfect"):
                tense = "i"
            if(tense == "perfect"):
                tense = "r"
            if(tense == "pluperfect"):
                tense = "l"
            if(tense == "future perfect"):
                tense = "t"
            if(tense == "future"):
                tense = "f"
            
                #voice
            if(voice == "active"):
                voice = "a"
            if(voice == "passive"):
                voice = "p"
            if(voice == "deponent"):
                voice = "d"
            
                #number
            if (number == "singular"):
                number = "s"
            if (number == "plural"):
                number = "p"
                # mood
            if(mood != "NULL"):
                posTag = posTag[:4] + mood + posTag[5:]
            # person
            if(person != "NULL"):
                posTag = posTag[:1] + person + posTag[2:]
            # tense
            if(tense != "NULL"):
                posTag = posTag[:3] + tense + posTag[4:]
            # voice
            if(voice != "NULL"):
                posTag = posTag[:5] + voice + posTag[6:]
            # number
            if(number != "NULL"):
                posTag = posTag[:2] + number + posTag[3:]
        if("a" in pos):
                #adj case

            #number
            if (number == "singular"):
                number = "s"
            if (number == "plural"):
                number = "p"
            if(number != "NULL"):
                posTag = posTag[:2] + number + posTag[3:]
            #gender 
            if(gender == "masculine"):
                gender = "m"
            if (gender == "feminine"):
                gender = "f"
            if (gender == "neuter"):
                gender = "n"
            if(gender != "NULL"):
                posTag = posTag[:6] + gender + posTag[7:]
            #degree
            if(degree == "positive"):
                degree = "p"
            if(degree == "comparative"):
                degree = "c"
            if(degree == "superlative"):
                degree = "s"
            if(degree != "NULL"):
                posTag = posTag[:8] + gender
        #set the pos to the wanted pos if it exists
        #change full words to their specific letters
        if(wanted_pos == "noun"):
            wanted_pos = "n"
        if(wanted_pos == "verb"):
            wanted_pos = "v"
        if(wanted_pos == "adjective"):
            wanted_pos = "a"
        if(wanted_pos == "adverb"):
            wanted_pos = "d"
        if(wanted_pos == "conjunction"):
            wanted_pos = "c"
        if(wanted_pos == "pronoun"):
            wanted_pos = "p"
        if(wanted_pos == "numeral"):
            wanted_pos = "m"
        if(wanted_pos == "interjection"):
            wanted_pos = "i"
        if(wanted_pos == "exclaimation"):
            wanted_pos = "e"
        if(wanted_pos == "punctuation"):
            wanted_pos = "u"
        #if the wanted pos is not null, change the posTag to match
        if(wanted_pos != "NULL"):
            posTag = wanted_pos + posTag[1:]
        #now actually find that word in parses 
        df = lemmaDF.loc[lemmaDF["id"] == id]
        for index, row in df.iterrows():
            fits = True
            if(wanted_pos == "v"):
                if(row["morph_code"][0] == "v" or row["morph_code"][0] == "t"):
                    fits = True
                else:
                    fits = False
                for i in range(1, 8):
                    char = posTag[i]
                    if(char != "-"):
                        #morph code is wrong or output is longer than old output or it has misc_features or alt dialect
                        if(row["morph_code"][i] != char or (output != "" and len(output)<len(row["bare_text"]))):
                            fits = False
                        #this is the case that we disclude it from results
            
            else:
                for i in range(8):
                    char = posTag[i]
                    if(char != "-"):
                        #morph code is wrong or output is longer than old output or it has misc_features or alt dialect
                        if(row["morph_code"][i] != char or (output != "" and len(output)<len(row["bare_text"]))):
                            fits = False
                        #this is the case that we disclude it from results
            if fits:
                output = row["bare_text"]

        #before repeating for other ids, return the output if it exists
        if(output != ""):
            return output
        
    return "ERROR: Nothing found with those parameters"
        
        
    
        

Run block 1 to initialize functions in kernel

get_words(str word) - returns a list of all words with that input in its list of parses

get_dict(str word, bool print_def = False) - returns all dictionary entries for a given word. 
    Gives definition if print_def is true

make_noun_dict(int[] ids, str outputfile = default path) - create a CSV that is a dictionary of all nouns in the database

getForm(word = "NULL", id = "NULL", case = "NULL", number = "NULL", gender = "NULL", mood = "NULL", person = "NULL", tense = "NULL", voice = "NULL", degree = "NULL", wanted_pos = "NULL")
    for voice, if deponent verb, put deponent
    Only put what you're looking for, ie 
    get_form(word = "Mater", case = "ablative", number = "singular") will return Matre (should)
        working on allowing "ablative" or "b" to be used for ablative

Current failure example: ID 33708

In [32]:
#make the charts - for nitai - from the words_csv
df = pd.read_csv("Dictionary_Dataframes/greek_words.csv", sep = "\t")
del df['Unnamed: 0']
#format for big file with all words: english grk_FPP sentiment
noun_FPP = []
noun_eng = []
noun_sentiment = []
verb_FPP = []
verb_eng = []
verb_sentiment = []
adj_FPP = []
adj_eng = []
adj_sentiment = []
adv_FPP = []
adv_eng = []
adv_sentiment = []
#iterate by id, not thru rows I think
for id in range(36488):
    pos = get_POS(id)
    for p in pos:
        #now we do it based on cases 
        if p == 'n':
            #noun case
            #get the fpp for the noun 
            fpp = get_form(id = id, number = "s", case = "n", wanted_pos= "n")
            #check if there is an issue and take plural if no singular
            if("ERROR" in fpp):
                fpp = get_form(id = id, number = "p", case = "n", wanted_pos= "n")
            #get the definition for the noun - acts as english 
            definition = get_def(id)
            #now fill out the basic chart 
            if(not "ERROR" in fpp):
                noun_FPP.append(fpp)
                noun_eng.append(definition)
                noun_sentiment.append("NULL")

        if p == 'v':
            fpp = get_form(id = id, number = "s", person = "1", tense = "p", voice = "a", mood = "i", wanted_pos= "v")
            #if issue, check in impersonal
            if("ERROR" in fpp):
                fpp = get_form(id = id, number = "s", person = "3", tense = "p", voice = "a", mood = "i", wanted_pos= "v")
            #now check deponent
            if("ERROR" in fpp):
                fpp = get_form(id = id, number = "s", person = "1", tense = "p", voice = "d", mood = "i", wanted_pos= "v")
            definition = get_def(id)
            if(not "ERROR" in fpp):
                verb_FPP.append(fpp)
                verb_eng.append(definition)
                verb_sentiment.append("NULL")
            #verb case
        if p == 'a':
            #adj case
            fpp = get_form(id = id, number = "s", case = "n", wanted_pos= "a", gender = "m", degree = "p")
            definition = get_def(id)
            #if no positive degree, get w/out
            if("ERROR" in fpp):
                fpp = get_form(id=id, number="s", case="n", wanted_pos="a", gender="m")
            #if no masc, get fem, then neutuer
            if("ERROR" in fpp):
                fpp = get_form(id=id, number="s", case="n", wanted_pos="a", gender="f")
            if("ERROR" in fpp):
                fpp = get_form(id=id, number="s", case="n", wanted_pos="a", gender="n")
            if(not "ERROR" in fpp):
                adj_FPP.append(fpp)
                adj_eng.append(definition)
                adj_sentiment.append("NULL")
        if p == 'd':
            #adv case
            fpp = get_form(id = id, wanted_pos= "d")
            definition = get_def(id)
            if(not "ERROR" in fpp):
                adv_FPP.append(fpp)
                adv_eng.append(definition)
                adv_sentiment.append("NULL")
        # if p == 'c':
        #     #conj case
        # if p == 'p':
        #     #pronoun case
        # if p == 'm':
        #     #numeral case
        # if p == 'i':
        #     #interjection case
        # if p == 'e':
        #     #exclaimation case
        # if p == 'u':
        #     #punctuation case
#now make the dataframes
noun_map = {
    "FPP": noun_FPP,
    "English": noun_eng,
    "Sentiment": noun_sentiment
}
noun_df = pd.DataFrame(noun_map)
noun_df.to_csv("greek_dictionary/nouns.csv", sep = "\t")

verb_map = {
    "FPP": verb_FPP,
    "English": verb_eng,
    "Sentiment": verb_sentiment
}
verb_df = pd.DataFrame(verb_map)
verb_df.to_csv("greek_dictionary/verbs.csv", sep = "\t")

adjective_map = {
    "FPP": adj_FPP,
    "English": adj_eng,
    "Sentiment": adj_sentiment
}
adjective_df = pd.DataFrame(adjective_map)
adjective_df.to_csv("greek_dictionary/adjectives.csv", sep = "\t")

adverb_map = {
    "FPP": adv_FPP,
    "English": adv_eng,
    "Sentiment": adv_sentiment
}
adverb_df = pd.DataFrame(adverb_map)
adverb_df.to_csv("greek_dictionary/adverbs.csv", sep = "\t")

        

In [30]:
#print(get_form(word = "ιοχεαιρα", number = "s", case = "a"))
print(get_id("περσις"))

[26042, 26043]
